In [1]:
import pandas as pd

data = pd.read_csv("Emotion_classify_Data.csv")

In [ ]:
import spacy

nlp=spacy.load("en_core_web_sm")

In [3]:
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)    
    return " ".join(filtered_tokens)


data["Comment"] = data['Comment'].apply(preprocess) 


data.Comment

corpus = [data["Comment"][i] for i in range(0,len(data))]    

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer()

### Target preprocess

In [6]:
target= data[["Emotion"]]

target = target.replace({"fear" : 0 , "anger" :1 , "joy":2})

target = target.astype(int)

C:\Users\PC1\AppData\Local\Temp\ipykernel_1344\3801369985.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  target = target.replace({"fear" : 0 , "anger" :1 , "joy":2})


### Split test/train 

In [7]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(corpus,target,test_size=0.20)

X_train_cv = v.fit_transform(X_train)
X_test_cv = v.transform(X_test)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  classification_report

model=RandomForestClassifier()
model.fit(X_train_cv, y_train)

y_pred = model.predict(X_test_cv)

print(classification_report(y_test, y_pred))

C:\Users\PC1\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


              precision    recall  f1-score   support

           0       0.92      0.91      0.92       378
           1       0.92      0.91      0.91       395
           2       0.92      0.94      0.93       415

    accuracy                           0.92      1188
   macro avg       0.92      0.92      0.92      1188
weighted avg       0.92      0.92      0.92      1188

